# NLP Pipeline (AWS/LOCAL)

Este notebook implementa un pipeline de NLP sobre reseñas de Amazon:

1) Ingesta (CSV) y limpieza  
2) Tokenización + stemming  
3) Vectorización TF‑IDF  
4) Sentiment analysis (**AWS Comprehend** o **VADER local**)  
5) Key Phrases + NER (**AWS Comprehend** o **entity candidates (regex simple; puede tener falsos positivos)**)  
6) Traducción INGLES --> ESPAÑOL (**AWS Translate**, opcional)

**Outputs**: se guardan en `outputs/`.


## 0) Configuración

**Kernel recomendado:** Python 3.11

### Dataset
- Coloca el archivo en `data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv`
- La columna de texto esperada es `reviewText`

### Modo AWS (opcional)
Si quieres ejecutar con AWS:
- Configura credenciales (`aws configure` o variables de entorno)
- Asegúrate de tener permisos para **Comprehend** y (opcionalmente) **Translate**


### CELDA 1: (Opcional) Dependencias

- Instala dependencias mínimas para ejecutar el notebook. **Opcional** si ya estás en una `.venv` con todo instalado.
- Incluye recursos de NLTK necesarios (tokenizer, stopwords, VADER).

**Ajustes rápidos:**
- Si no vas a traducir en local, puedes omitir `transformers/sentencepiece/torch`.


In [ ]:
# CELDA 1: (Opcional) Dependencias
# Si ya tienes un entorno del proyecto listo, puedes omitir esta celda.

%pip -q install boto3 pandas numpy nltk scikit-learn transformers sentencepiece torch

import nltk
for pkg in ["punkt", "punkt_tab", "stopwords", "vader_lexicon"]:
    nltk.download(pkg, quiet=True)

print("Dependencias listas")


### CELDA 2: Imports + configuración (AWS/local)

- Configura el entorno (AWS/local), rutas y límites de procesamiento. Esta celda define los *flags* que controlan costos/tiempo.
- Recomendación: dejar `ENABLE_TRANSLATE=0` por defecto y activarlo solo cuando lo necesites.

**Ajustes rápidos:**
- `MAX_DOCS` (número máximo de documentos a procesar)
- `MAX_TRANSLATE_DOCS` (límite de documentos a traducir)
- `RAW_TEXT_MAX_CHARS` (truncado por documento)
- `CSV_NROWS` (filas a leer del CSV)
- `ENABLE_TRANSLATE` (0/1 para activar traducción)
- Variables de entorno: `AWS_REGION`, `NLP_PROVIDER`, `MAX_DOCS`, `ENABLE_TRANSLATE`, `MAX_TRANSLATE_DOCS`, `CSV_NROWS`, `RAW_TEXT_MAX_CHARS`.


In [ ]:
# CELDA 2: Imports + configuración (AWS/local)
import os, json, time, re, logging
from pathlib import Path

import nltk
import numpy as np
import pandas as pd
import boto3
from IPython.display import display

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# --- Parámetros (puedes cambiarlos aquí o por variables de entorno) ---
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")
PROVIDER = os.getenv("NLP_PROVIDER", "auto").lower()  # auto | aws | local
MAX_DOCS = int(os.getenv("MAX_DOCS", "5000"))

# Traducción puede generar costos si usas AWS Translate.
# Por defecto está apagada (costo 0).
# Para habilitar la traducción solo cambia el valor de 0 a 1
ENABLE_TRANSLATE = os.getenv("ENABLE_TRANSLATE", "0") == "1"
MAX_TRANSLATE_DOCS = int(os.getenv("MAX_TRANSLATE_DOCS", "200")) # Puedes modificar este valor, si notas que consume mucha cpu

# Lectura/truncado del dataset (impacta costo/tiempo)
# Si notas que consume mucha cpu puedes reducir la cantidad de rows
CSV_NROWS = int(os.getenv("CSV_NROWS", "100000"))
RAW_TEXT_MAX_CHARS = int(os.getenv("RAW_TEXT_MAX_CHARS", "1000"))

# SI QUIERES UTILIZAR OTRO DATASET DIFERENTE, TOMA EN CUENTA ESTO:
# - DATA_PATH: ruta al archivo .csv (puede ser cualquier dataset en formato CSV). Ej: my_dataset.csv 
# - TEXT_COLUMN: nombre de la columna dentro del csv que contiene el texto a analizar. Ej: text
# Nota: Si tu CSV usa otro separador o encoding, ajusta la lectura en la celda de carga
# (por ejemplo: sep=";" o encoding="latin-1").
DATA_PATH = Path(os.getenv("DATA_PATH", "data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv"))
TEXT_COLUMN = os.getenv("TEXT_COLUMN", "reviewText")

# --- Detección de credenciales AWS ---
session = boto3.Session(region_name=AWS_REGION)
_creds = session.get_credentials()
has_aws_creds = bool(_creds and _creds.access_key)

use_aws = PROVIDER == "aws" or (PROVIDER == "auto" and has_aws_creds)
print(f"Provider: {'AWS' if use_aws else 'Local'} | region={AWS_REGION} | MAX_DOCS={MAX_DOCS}")

comprehend = session.client("comprehend") if use_aws else None
translate = session.client("translate") if use_aws else None

def ensure_nltk_resource(resource: str, download_name: str | None = None):
    try:
        nltk.data.find(resource)
    except LookupError:
        nltk.download(download_name or resource.split("/")[-1])

# Tokenizer resources
ensure_nltk_resource("tokenizers/punkt", "punkt")
ensure_nltk_resource("tokenizers/punkt_tab", "punkt_tab")  # por compatibilidad

def chunked(seq, size):
    for i in range(0, len(seq), size):
        yield i, seq[i:i+size]

# --- HuggingFace (traducción local) ---
HF_HOME = os.getenv("HF_HOME", str(Path(".hf_cache").resolve()))
os.environ["HF_HOME"] = HF_HOME
HF_TRANSLATE_MODEL = os.getenv("HF_TRANSLATE_MODEL", "Helsinki-NLP/opus-mt-en-es")



## CELDA 2.1: Traducción local INGLES --> ESPAÑOL (OPCIONAL) (Hugging Face · MarianMT)

***Ejecuta este helper SOLO si:***
- ENABLE_TRANSLATE=1 y no hay AWS Translate disponible.
- Si decides forzar el modo local para traducción. 
- Requiere: transformers + sentencepiece + torch.


In [ ]:
# Helper: Traducción local INGLE A ESPAÑOL (HuggingFace / MarianMT)

from typing import List, Optional

def hf_translate_en_es(
    texts: List[str],
    batch_size: int = 16,
    max_length: int = 256,
    device: Optional[str] = "cpu",
) -> List[str]:
    """Traduce una lista de textos de ingles a español usando MarianMT.

    Parámetros:
    - texts: lista de textos (strings).
    - batch_size: tamaño del lote (más alto = más rápido, pero más RAM).
    - max_length: longitud máxima de salida (reduce si quieres velocidad).
    - device: "cpu" o "cuda" si tienes GPU.

    Devuelve:
    - lista de traducciones en el mismo orden.
    """
    try:
        from transformers import MarianMTModel, MarianTokenizer
    except ModuleNotFoundError as e:
        raise ModuleNotFoundError(
            "Faltan dependencias para traducción local. Instala: "
            "pip install transformers sentencepiece torch"
        ) from e

    tokenizer = MarianTokenizer.from_pretrained(HF_TRANSLATE_MODEL)
    model = MarianMTModel.from_pretrained(HF_TRANSLATE_MODEL)
    model.to(device)

    out: List[str] = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch = [("" if t is None else str(t)) for t in batch]

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        translated = model.generate(**inputs, max_length=max_length)
        out.extend(tokenizer.batch_decode(translated, skip_special_tokens=True))

    return out


## 1) Preprocesamiento (Lexer)
**Extracción CSV --> Limpieza --> Tokenización --> TF‑IDF**


### CELDA 3: Logging 

- Inicializa un logger simple y un colector de eventos exportable a JSON para auditoría del pipeline.
- Esta celda simplemente tiene la finalidad de crear un sistema logging donde se registre todo los procesos que sucedan en el notebook

In [ ]:
# CELDA 3: Logging 
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)
log = logging.getLogger("nlp-pipeline")

# Aca se guardan los eventos
_events = []

# Imprime los logs
def log_event(level: str, message: str, **meta):
    level = level.upper()
    getattr(log, level.lower(), log.info)(message)
    _events.append({"ts": time.time(), "level": level, "message": message, **meta})

# Guarda el evento en _events 
def save_logs(path: Path = OUTPUT_DIR / "pipeline_logs.json"):
    path = Path(path)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(_events, f, ensure_ascii=False, indent=2)
    print(f"Logs guardados en: {path}")

# Evento inicial, para saber que inicio el pipeline
log_event("INFO", "Notebook inicializado")


### CELDA 4: Extracción de texto 

- Carga el dataset desde CSV y extrae la columna de texto. Y devuelve una lista llamada "raw_texts".

**Ajustes rápidos:**
- `MAX_DOCS` (número máximo de documentos a procesar)
- `RAW_TEXT_MAX_CHARS` (truncado por documento)
- `CSV_NROWS` (filas a leer del CSV)
- Ajusta `DATA_PATH` y `TEXT_COLUMN` si usas otro dataset.


In [ ]:
# CELDA 4: Extracción de texto

# Función que utiliza los defaults definidos en la CELDA 2
def extract_csv_reviews(
    path: Path = DATA_PATH,
    text_col: str = TEXT_COLUMN,
    nrows: int = CSV_NROWS,
    limit: int = MAX_DOCS,
):
    path = Path(path)
    if not path.exists(): # Verifica que el dataset exista
        raise FileNotFoundError(
            f"El dataset no se encontro en {path}. "
            "Colócalo en la carpeta data/ o define DATA_PATH."
        )

    # Si tu .csv no es UTF-8 puede fallar, considera modificar el encoding 
    # a latin-1, cp1252 o el que se adapte a tu csv
    df = pd.read_csv(path, nrows=nrows, encoding="utf-8") 
    if text_col not in df.columns: # Verifica que la columna exista, verificar CELDA 2, si ocurre algun error.
        raise KeyError(f"Columna '{text_col}' no existe. Columnas disponibles: {list(df.columns)[:20]}")

    texts = (
        df[text_col]
        .dropna() # elimina filas sin texto       
        .astype(str) # esto garantiza que todo sea tipo string
        .str[:RAW_TEXT_MAX_CHARS]   # límite de longitud para Comprehend/Translate
        .tolist()
    )
    texts = texts[:limit] # Limita la cantidad a solo los descritos en "MAX_DOCS" 

    log_event("INFO", "Dataset cargado", path=str(path), nrows=int(df.shape[0]), docs=len(texts))
    return texts

# Crea la lista raw_texts y muestra los priemros 250 caracteres
raw_texts = extract_csv_reviews()
print("Ejemplo:", raw_texts[0][:250], "...")


### CELDA 5: Limpieza y normalización

- Normaliza el texto para el resto del pipeline local.

**Ajustes rápidos:**
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# CELDA 5: Limpieza y normalización

def clean_normalize(text: str) -> str:
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # URLs
    text = text.strip()
    text = re.sub(r"\s+", " ", text)               # Espacios múltiples
    return text

# Guardamos 2 versiones:
# - raw_texts: para NER heurístico local o AWS NER
# - cleaned_texts: para análisis mas general
cleaned_texts = [clean_normalize(t).lower() for t in raw_texts]

# Filtrar textos vacíos o muy cortos 
valid_idx = [i for i, t in enumerate(cleaned_texts) if len(t) >= 10]
raw_texts = [raw_texts[i] for i in valid_idx]
cleaned_texts = [cleaned_texts[i] for i in valid_idx]

log_event("INFO", "Limpieza completada", docs=len(cleaned_texts))
print(f"Textos válidos: {len(cleaned_texts)}")


### CELDA 6: Tokenización + stemming

- Convierte los textos limpios en una versión mas normalizada, preparandolos para el proceso de Term Frequency-Inverse Document Frequency o TF-IDF.
- Basicamente con stemming lo que conseguimos es reducir la palabras a su raiz. Ej: learning a learn.

**Ajustes rápidos:**
- Nota: Este paso se usa principalmente para el modo local (TF‑IDF).
- Si solo quieres probar AWS Comprehend, puedes saltar TF‑IDF y este paso.
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# CELDA 6: Tokenización + stemming

from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Aca se crea el stemmer o reductor, que reducira las palabras.
stemmer = SnowballStemmer("english")

def tokenize_stem(texts):
    """Tokeniza y aplica stemming (Snowball) a una lista de textos.

    - Entrada: lista de strings (idealmente ya limpiados).
    - Salida: lista de strings tokenizados/stemmeados.

    **Tip portafolio:** si quieres acelerar iteraciones, reduce `MAX_DOCS` en la celda de Configuración.
    """
    tokenized = []
    for text in texts:
        tokens = word_tokenize(text)
        stemmed = [stemmer.stem(tok) for tok in tokens if len(tok) > 2]
        tokenized.append(" ".join(stemmed))
    return tokenized

# Genera "tokenized_texts", que se usa en la celda de TF-IDF
tokenized_texts = tokenize_stem(cleaned_texts)
log_event("INFO", "Tokenización + stemming completado", docs=len(tokenized_texts))
print("Ejemplo:", tokenized_texts[0][:120], "...")


### CELDA 7: Vectorización TF‑IDF

- Convierte los textos ya limpios y normalizados en una una matriz TF‑IDF.

**Ajustes rápidos:**
- `TFIDF_MAX_FEATURES` (tamaño del vocabulario TF‑IDF)
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# CELDA 7: Vectorización TF‑IDF
# Este bloque construye features para el modo local (sin AWS).
# Ajusta `TFIDF_MAX_FEATURES` si quieres:
# - más calidad (↑ features) o
# - más velocidad/memoria (↓ features).

from sklearn.feature_extraction.text import TfidfVectorizer

TFIDF_MAX_FEATURES = int(os.getenv("TFIDF_MAX_FEATURES", "5000"))

tfidf_model = TfidfVectorizer(
    max_features=TFIDF_MAX_FEATURES, # Limita el vocabulario a la cantidad definida
    stop_words="english", # Elimina las stop words
    ngram_range=(1, 2),
)

# Determina la rareza en pesos de las palabras y convierte cada documento en un vector
tfidf_matrix = tfidf_model.fit_transform(tokenized_texts)

# Verifica mostrando los primeros terminos del vocabulario
top_terms = tfidf_model.get_feature_names_out()[:10]
log_event("INFO", "TF‑IDF construido", shape=str(tfidf_matrix.shape), top_terms=top_terms.tolist())
print(f"TF‑IDF listo: shape={tfidf_matrix.shape} | Ejemplo top términos: {top_terms.tolist()}")


## 2) Parsers 
**Sentiment --> Key Phrases --> NER --> Translate (opcional)**


### CELDA 8: Sentiment analysis (AWS Comprehend o VADER local)

- Analiza sentimiento de dos formas: 
- usa AWS Comprehend.
- VADER (NLTK) en modo local.

**Ajustes rápidos:**
- `batch_size` (tamaño de lote para llamadas AWS)
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# CELDA 8: Sentiment analysis (AWS Comprehend o VADER local)

def sentiment_local(texts):
    from nltk.sentiment import SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    out = []
    for t in texts:
        score = sia.polarity_scores(t)
        compound = score["compound"]
        if compound >= 0.05:
            label = "POSITIVE"
        elif compound <= -0.05:
            label = "NEGATIVE"
        else:
            label = "NEUTRAL"

        out.append({
            "sentiment": label,
            # "confianzas" aproximadas para mantener un schema similar al de comprehend
            # 
            "positive": round(max(compound, 0.0), 4),
            "negative": round(max(-compound, 0.0), 4),
        })
    return out

def sentiment_aws(texts):
    if comprehend is None:
        raise RuntimeError("Comprehend no está inicializado (use_aws=False).")

    out = [None] * len(texts)
    batch_size = 25  # límite de BatchDetectSentiment(comprehend normalmente permite hasta 25)
    for offset, batch in chunked(texts, batch_size):
        resp = comprehend.batch_detect_sentiment(TextList=batch, LanguageCode="en")

        for r in resp.get("ResultList", []):
            idx = offset + r["Index"]
            ss = r.get("SentimentScore", {})
            out[idx] = {
    # - La etiqueta `sentiment` NO cambia, porque la decide el umbral sobre `compound`
    #   (>= 0.05 positivo, <= -0.05 negativo, si no neutral).
    # - Pero SÍ cambia la "intensidad" que reportas y cualquier análisis posterior que use
    #   estos campos (rankings, filtros por confianza, promedios, gráficos).
    # - Si quieres cambiar cuántos textos caen en POS/NEG/NEU, ajusta los umbrales del compound, no estos campos.
    # Ejemplo: compound=0.72 -> positive=0.72, negative=0.00 (fuerte positivo)
    # compound=-0.40 -> positive=0.00, negative=0.40 (negativo moderado)

                "sentiment": r.get("Sentiment", "NEUTRAL"),
                "positive": round(ss.get("Positive", 0.0), 4),
                "negative": round(ss.get("Negative", 0.0), 4),
            }

        # Si hubo errores, dejamos neutral
        for e in resp.get("ErrorList", []):
            idx = offset + e["Index"]
            out[idx] = {"sentiment": "NEUTRAL", "positive": 0.0, "negative": 0.0}

        time.sleep(0.05)  # pequeño respiro (rate limits)
    return out

# Si estan activas las credenciales para aws usa comprehend, sino utiliza VADER de forma local
sentiments = sentiment_aws(cleaned_texts) if use_aws else sentiment_local(cleaned_texts)

sent_counts = pd.Series([s["sentiment"] for s in sentiments]).value_counts().to_dict()
log_event("INFO", "Sentiment completado", provider=("aws" if use_aws else "local"), counts=sent_counts)
print(" Sentiment:", sent_counts)


### CELDA 9: Key Phrases + NER

- Extrae key phrases y entidades: AWS Comprehend o heurísticas locales.
- Nota: Para NER en AWS se usa `raw_texts` para preservar mayúsculas.

**Ajustes rápidos:**
- `batch_size` (tamaño de lote para llamadas AWS)
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# CELDA 9: Key Phrases + NER (AWS Comprehend o heurística local)
# - Modo AWS: usa BatchDetectKeyPhrases y BatchDetectEntities (25 docs por batch).
# - Modo local: key phrases via TF‑IDF + entidades con heurística regex (simple, sin costo).
#
# Para mejorar la calidad de NER en AWS, usamos `raw_texts` (preserva mayúsculas).
# Nota: Comprehend tiene límites por documento (por eso truncamos en la lectura del CSV).

AWS_COMPREHEND_BATCH = int(os.getenv("AWS_COMPREHEND_BATCH", "25"))
AWS_ENTITY_MIN_SCORE = float(os.getenv("AWS_ENTITY_MIN_SCORE", "0.80"))

def key_phrases_local(matrix, model, top_k=3):
    """Top-k términos por documento usando pesos TF‑IDF (aprox. key phrases local)."""
    feature_names = np.array(model.get_feature_names_out())
    out = []
    for i in range(matrix.shape[0]):
        row = matrix[i]
        if row.nnz == 0:
            out.append([])
            continue
        vals = row.toarray().ravel()
        idxs = vals.argsort()[-top_k:][::-1]
        out.append(feature_names[idxs].tolist())
    return out

def entities_local(texts_raw, max_entities=5):
    """Heurística simple: extrae candidatos 'tipo entidad' por patrones.
    No reemplaza un modelo NER real, pero sirve como fallback reproducible.
    """
    out=[]
    for t in texts_raw:
        candidates = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*\b", t)
        seen=set(); uniq=[]
        for c in candidates:
            c = c.strip()
            if len(c) < 3:
                continue
            if c not in seen:
                seen.add(c)
                uniq.append(c)
        out.append(uniq[:max_entities])
    return out

def key_phrases_aws(texts):
    """Key phrases con AWS Comprehend (batch)."""
    if comprehend is None:
        raise RuntimeError("Comprehend no está inicializado (use_aws=False).")

    out = [[] for _ in range(len(texts))]
    for offset, batch in chunked(texts, AWS_COMPREHEND_BATCH):
        resp = comprehend.batch_detect_key_phrases(TextList=batch, LanguageCode="en")

        for r in resp.get("ResultList", []):
            idx = offset + r["Index"]
            phrases = sorted(r.get("KeyPhrases", []), key=lambda x: x.get("Score", 0.0), reverse=True)
            out[idx] = [p["Text"] for p in phrases[:3]]

        time.sleep(0.05)
    return out

def entities_aws(texts, min_score=AWS_ENTITY_MIN_SCORE):
    """Entidades con AWS Comprehend (batch)."""
    if comprehend is None:
        raise RuntimeError("Comprehend no está inicializado (use_aws=False).")

    out = [[] for _ in range(len(texts))]
    for offset, batch in chunked(texts, AWS_COMPREHEND_BATCH):
        resp = comprehend.batch_detect_entities(TextList=batch, LanguageCode="en")

        for r in resp.get("ResultList", []):
            idx = offset + r["Index"]
            ents = [e["Text"] for e in r.get("Entities", []) if e.get("Score", 0.0) >= min_score]
            # unique preserve order
            seen=set(); uniq=[]
            for e in ents:
                if e not in seen:
                    seen.add(e); uniq.append(e)
            out[idx] = uniq[:8]

        time.sleep(0.05)
    return out

n_docs = len(raw_texts)

# Inicializa ambos para mantener schema consistente
entities_por_doc = [[] for _ in range(n_docs)]              # NER real (solo AWS)
entity_candidates_por_doc = [[] for _ in range(n_docs)]     # heurístico (solo local)

if use_aws:
    key_phrases_por_doc = key_phrases_aws(raw_texts)
    entities_por_doc = entities_aws(raw_texts)
    ner_provider = "aws_comprehend"
else:
    key_phrases_por_doc = key_phrases_local(tfidf_matrix, tfidf_model)
    entity_candidates_por_doc = entities_local(raw_texts)
    ner_provider = "local_regex_candidates"

log_event("INFO", "Key Phrases + NER completado", provider=ner_provider)

def pick_reasonable_example(raw_texts, candidates):
    bad_singletons = {"Waste", "Great", "Good", "Bad", "Terrible", "Awesome", "Perfect", "Money"}
    for i, ents in enumerate(candidates):
        if not ents:
            continue
        if len(ents) >= 2:
            return i
        if len(ents) == 1 and ents[0] not in bad_singletons and len(ents[0]) >= 6:
            return i
    return None

print("Ejemplo key phrases:", key_phrases_por_doc[0])

if use_aws:
    idx = next((i for i, ents in enumerate(entities_por_doc) if ents), None)
    print("Ejemplo Entities (AWS NER):", entities_por_doc[idx] if idx is not None else [])
    if idx is not None:
        print("Texto ejemplo:", raw_texts[idx][:200], "...")
else:
    idx = pick_reasonable_example(raw_texts, entity_candidates_por_doc)
    print("Ejemplo Entity candidates (local heurístico):", entity_candidates_por_doc[idx] if idx is not None else [])
    if idx is not None:
        print("Texto ejemplo:", raw_texts[idx][:200], "...")
    else:
        print("No se encontraron candidatos (normal si no hay nombres propios o el texto no los marca).")



### Export: key phrases + entities por documento

- Exporta resultados intermedios a `outputs/` para inspeccionarlos sin necesidad de volver a ejecutar el notebook.


In [ ]:
# Export: key phrases + entities por documento
export_path = OUTPUT_DIR / "keyphrases_entities.csv"

# Crea un dataframe con los resultados
df_kp = pd.DataFrame({
    "doc_id": range(len(cleaned_texts)),
    "text_clean": cleaned_texts,
    "key_phrases": ["; ".join(k) for k in key_phrases_por_doc],
    "entities_aws": ["; ".join(e) for e in entities_por_doc],  # solo AWS
    "entity_candidates_local": ["; ".join(e) for e in entity_candidates_por_doc],  # solo local
    "ner_provider": [ner_provider] * len(cleaned_texts),
})
df_kp.to_csv(export_path, index=False) # Guarda el dataframe
print(f"Exportado: {export_path}") # Confirma la exportación 

### CELDA 10: Traducción INGLES --> ESPAÑOL (AWS Translate o HuggingFace local, opcional)

- Traduce de INGLES A ESPAÑOL. Si hay AWS y lo habilitas, usa AWS Translate; si no, usa Hugging Face (MarianMT) en local.
- Por costo, viene desactivado por defecto. Traduce solo un subconjunto con `MAX_TRANSLATE_DOCS`.

**Ajustes rápidos:**
- `MAX_TRANSLATE_DOCS` (límite de documentos a traducir)
- `ENABLE_TRANSLATE` (0/1 para activar traducción)
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# CELDA 10: Traducción INGLES --> ESPAÑOL

if "raw_texts" not in globals():
    raise NameError("raw_texts no existe. Ejecuta primero las celdas de carga de datos (CELDA 4).")

translations = [""] * len(raw_texts)

# Ajustes para HuggingFace local (si aplica)
HF_BATCH_SIZE = int(os.getenv("HF_BATCH_SIZE", "16"))
HF_MAX_LENGTH = int(os.getenv("HF_MAX_LENGTH", "128"))

if ENABLE_TRANSLATE:
    n = min(MAX_TRANSLATE_DOCS, len(raw_texts))
    texts_subset = raw_texts[:n]

    if use_aws and translate is not None:
        out = []
        for i, text in enumerate(texts_subset, start=1):
            if i % 50 == 0 or i == 1:
                print(f"Traduciendo (AWS)... {i}/{n}")
            try:
                resp = translate.translate_text(
                    Text=text,
                    SourceLanguageCode="en",
                    TargetLanguageCode="es",
                )
                out.append(resp["TranslatedText"])
            except Exception as e:
                out.append("")  # no ensuciamos texto con 'ERROR: ...'
                log_event("WARN", "Fallo AWS Translate", idx=i, error=str(e)[:120])

        translations[:n] = out
        log_event("INFO", "Traducción completada", provider="aws", translated_docs=n)
        print(f"Traducciones ES completadas (AWS): {n}")

    else:
        print("AWS no disponible. Usando HuggingFace (local) para traducción EN→ES…")
        print("(La primera vez descargará el modelo y puede tardar un poco.)")
        
        # Llama al helper "hf_translate_en_es" que definimos en la celda 2.1
        # Nota: `hf_translate_en_es` está definido en una celda previa.
        out = hf_translate_en_es(
            texts_subset,
            batch_size=HF_BATCH_SIZE,
            max_length=HF_MAX_LENGTH,
            device="cpu",
        )
        translations[:n] = out
        log_event("INFO", "Traducción completada", provider="hf_local", translated_docs=n)
        print(f"Traducciones completadas (HF local): {n}")

# En caso de que la traducción este desactivada
else:
    log_event("INFO", "Traducción omitida (ENABLE_TRANSLATE=0)")
    print("ℹTraducción omitida. (ENABLE_TRANSLATE=0)")


### Export: traducciones (Si esta habilitada)

- Exporta resultados intermedios a `outputs/` para inspección rápida y depuración.

**Ajustes rápidos:**
- `ENABLE_TRANSLATE` (0/1 para activar traducción)
- `MAX_DOCS` (número máximo de documentos a procesar; se controla en la celda de Configuración)


In [ ]:
# Export: traducciones
# Nota: Si ENABLE_TRANSLATE=0, la columna de traducción quedará vacía.
export_path = OUTPUT_DIR / "translations_en_es.csv" # Define la ruta

df_tr = pd.DataFrame({
    "doc_id": range(len(raw_texts)), # Id del documenyo 
    "text_raw_en": raw_texts, # Texto original en ingles
    "text_clean_en": cleaned_texts, # Texto limpio en ingles
    "text_translated_es": translations, # Texto traducido a español
})
df_tr.to_csv(export_path, index=False)
print(f"Exportado: {export_path}")

## 3) Export final del pipeline

Genera un CSV/JSON con todos los artefactos en un solo dataset:
- texto raw + texto limpio
- sentiment (+ scores)
- key phrases + entities
- TF‑IDF top term
- traducción (si está activada)


### CELDA 11: Export final 

- Guarda todos los outputs en un único dataset y los logs, exportandolos en csv + json.


In [ ]:
# CELDA 11: Export final 
n_docs = len(raw_texts)

# Top término TF‑IDF por documento
try:
    feature_names = tfidf_model.get_feature_names_out()
    tfidf_top = []
    for i in range(n_docs):
        row = tfidf_matrix[i]
        if row.nnz == 0:
            tfidf_top.append("")
        else:
            j = int(row.toarray().ravel().argmax())
            tfidf_top.append(feature_names[j])
except Exception:
    tfidf_top = [""] * n_docs

results = pd.DataFrame({
    "doc_id": range(n_docs),
    "text_raw": raw_texts[:n_docs],
    "text_clean": cleaned_texts[:n_docs],
    "sentiment": [s["sentiment"] for s in sentiments[:n_docs]],
    "conf_pos": [s["positive"] for s in sentiments[:n_docs]],
    "conf_neg": [s["negative"] for s in sentiments[:n_docs]],
    "tfidf_top": tfidf_top[:n_docs],
    "key_phrases": ["; ".join(k) for k in key_phrases_por_doc[:n_docs]],
    "entities_aws": ["; ".join(e) for e in entities_por_doc[:n_docs]],
    "entity_candidates_local": ["; ".join(e) for e in entity_candidates_por_doc[:n_docs]],
    "ner_provider": [ner_provider] * n_docs,
    "trad_es": translations[:n_docs],
})
# rutas de exportación
csv_path = OUTPUT_DIR / "pipeline_results.csv"
json_path = OUTPUT_DIR / "pipeline_results.json"

results.to_csv(csv_path, index=False)
results.to_json(json_path, orient="records", indent=2, force_ascii=False)

save_logs() # Guarda los logs

print(f"Export final listo: {csv_path} | {json_path}")
print("Preview:")
display(results.head(5))
